In [2]:
import sys, os, time
from pathlib import Path
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

import scipy.io


def humansize(nbytes):
    '''From https://stackoverflow.com/questions/14996453/python-libraries-to-calculate-human-readable-filesize-from-bytes'''
    suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
    i = 0
    while nbytes >= 1024 and i < len(suffixes)-1:
        nbytes /= 1024.
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

ds_path = '/mnt/exdisk1/danigil/datasets/imagenet'

path_imagenet_val_dataset = Path(os.path.join(ds_path, "processed")) # path/to/data/
dir_images = Path(os.path.join(ds_path, "ILSVRC2012_img_val")) # path/to/images/directory
path_labels = Path(os.path.join(ds_path, "ILSVRC2012_validation_ground_truth.txt"))
path_synset_words = Path(os.path.join(ds_path, "synset_words.txt"))
path_meta = Path(os.path.join(ds_path, "meta.mat"))

image_paths = sorted(glob(str(dir_images/"*")))
n_images = len(image_paths)

def create_imagenet_preprocessed_y():
    meta = scipy.io.loadmat(str(path_meta))
    original_idx_to_synset = {}
    synset_to_name = {}

    for i in range(1000):
        ilsvrc2012_id = int(meta["synsets"][i,0][0][0][0])
        synset = meta["synsets"][i,0][1][0]
        name = meta["synsets"][i,0][2][0]
        original_idx_to_synset[ilsvrc2012_id] = synset
        synset_to_name[synset] = name

    synset_to_keras_idx = {}
    keras_idx_to_name = {}
    with open(str(path_synset_words), "r") as f:
        for idx, line in enumerate(f):
            parts = line.split(" ")
            synset_to_keras_idx[parts[0]] = idx
            keras_idx_to_name[idx] = " ".join(parts[1:])

    convert_original_idx_to_keras_idx = lambda idx: synset_to_keras_idx[original_idx_to_synset[idx]]

    with open(str(path_labels),"r") as f:
        y_val = f.read().strip().split("\n")
        y_val = np.array([convert_original_idx_to_keras_idx(int(idx)) for idx in y_val])

    return y_val

In [3]:
y_val = create_imagenet_preprocessed_y()

In [4]:
y_val.shape

(50000,)

In [5]:
n_images

50000

In [8]:
import tensorflow as tf

ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dir_images,
    labels=list(y_val),
    label_mode='int',

    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(256, 256),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 50000 files belonging to 1000 classes.


2024-08-16 17:18:06.817950: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [1]:
from imagenet12_util import ret_imagnet12_tf_ds

ds = ret_imagnet12_tf_ds((224, 224), interpolation='bicubic', batch_size=32)

ds.save('/mnt/exdisk1/danigil/datasets/imagenet/processed/imagenet12_tf_ds')

2024-08-16 20:16:02.852269: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-16 20:16:02.875045: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 20:16:02.875067: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 20:16:02.875081: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 20:16:02.879504: I tensorflow/core/platform/cpu_feature_g

Found 50000 files belonging to 1000 classes.


2024-08-16 20:16:04.575019: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-16 20:16:04.591617: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-16 20:16:04.594465: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [1]:
import tensorflow as tf

from imagenet12_util import ret_imagnet12_tf_ds

def ret_model_preprocessing_by_name(model_name, zoo_lib="torch"):
    if zoo_lib=='torch':
        return
    
    if zoo_lib=="keras":
        keras_models_map = {
                "MobileNet": 'mobilenet', 
                "MobileNetV2": 'mobilenet_v2',
                "MobileNetV3Small": 'mobilenet_v3',
                "MobileNetV3Large": 'mobilenet_v3',
                "NASNetMobile": 'nasnet',
                "DenseNet121": 'densenet',
                "EfficientNetV2B0": 'efficientnet_v2',
                "EfficientNetV2B1": 'efficientnet_v2',

                "Xception": 'xception',
                "ResNet50": 'resnet',
                "ResNet50V2": 'resnet_v2',
                "ResNet101": 'resnet',
                "ResNet101V2": 'resnet_v2',
                "ResNet152": 'resnet',
                "ResNet152V2": 'resnet_v2',

                "InceptionV3": 'inception_v3',
                "InceptionResNetV2": 'inception_resnet_v2',
                "DenseNet169": 'densenet',
                "DenseNet201": 'densenet',
                "NASNetLarge": 'nasnet',

                "EfficientNetV2B2": 'efficientnet_v2',
                "EfficientNetV2B3": 'efficientnet_v2',
                "EfficientNetV2S": 'efficientnet_v2',
                "EfficientNetV2M": 'efficientnet_v2',

                "ConvNeXtTiny": 'convnext',
                "ConvNeXtSmall": 'convnext',
                "ConvNeXtBase": 'convnext',
        }  

        import tensorflow.keras.applications
        ret_class = getattr(tensorflow.keras.applications, keras_models_map.get(model_name, None), None)
        if not ret_class:
            raise Exception(f"ret_keras_model_by_name | model_name {model_name} not found")

        return ret_class.preprocess_input


model_name = "ResNet50"

preprocessing_func = ret_model_preprocessing_by_name(model_name, zoo_lib="keras")

with tf.device('/device:GPU:0'):
    ds = ret_imagnet12_tf_ds((224, 224), interpolation='bilinear', batch_size=32) \
        .map(lambda x, y: (preprocessing_func(x), y), num_parallel_calls=tf.data.AUTOTUNE) \
        

    # for x in ds.take(1):
        # print(x.shape)
        # print(preprocessing_func(x[0]))
    model = tf.keras.applications.ResNet50()

    y_pred = model.predict(ds.take(100))
    # model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # y_preds = model.evaluate(ds)

2024-08-17 16:05:52.312431: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-17 16:05:52.335173: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 16:05:52.335192: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 16:05:52.335207: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-17 16:05:52.339499: I tensorflow/core/platform/cpu_feature_g

Found 50000 files belonging to 1000 classes.


2024-08-17 16:05:56.043855: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-08-17 16:05:56.115827: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


100/100 [==============================] - 11s 96ms/step


In [3]:
type(y_pred)

numpy.ndarray

In [6]:
y_pred.shape

(50000, 1000)

In [7]:
y_val_one_hot.shape

(50000, 1000)

In [5]:
y_val.shape

(50000,)

In [4]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

def top_k_accuracy(y_true, y_pred, k=1, tf_enabled=True):
    '''
    Calculates top_k accuracy of predictions. Expects both y_true and y_pred to be one-hot encoded.
    numpy implementation is from: https://github.com/chainer/chainer/issues/606
    '''

    if tf_enabled:
        argsorted_y = tf.argsort(y_pred)[:,-k:]
        matches = tf.cast(tf.math.reduce_any(tf.transpose(argsorted_y) == tf.argmax(y_true, axis=1, output_type=tf.int32), axis=0), tf.float32)
        return tf.math.reduce_mean(matches).numpy()
    else:
        argsorted_y = np.argsort(y_pred)[:,-k:]
        return np.any(argsorted_y.T == y_true.argmax(axis=1), axis=0).mean()


y_val_one_hot = to_categorical(y_val, 1000)

acc = top_k_accuracy(y_val_one_hot, y_pred, k=1)
acc

2024-08-16 19:02:49.820409: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


0.70258

In [1]:
import os
os.environ["TF_CPP_MIN_VLOG_LEVEL"] = "2"

import tensorflow as tf

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2024-08-17 16:02:31.195174: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-17 16:02:31.217703: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 16:02:31.217723: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 16:02:31.217736: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-17 16:02:31.221996: I tensorflow/core/platform/cpu_feature_g

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17554185505614704514
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 8450211840
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5848602579826262967
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2024-08-17 16:02:32.415381: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-17 16:02:32.433400: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-17 16:02:32.436700: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf